In [7]:
from src.utils import *

In [14]:
# Load required libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import fiona
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from shapely.ops import unary_union
from unidecode import unidecode
import glob
import csv
from datetime import datetime
import dask.dataframe as dd
import dask_geopandas as dg
from dask.distributed import Client
import gc


In [15]:
def write_log(log_path, log_entry):
    # Check if the file exists to determine if headers need to be written
    file_exists = os.path.isfile(log_path)

    # Open the log file in append mode
    with open(log_path, 'a', newline='') as csvfile:
        log_writer = csv.writer(csvfile)

        # Write the header if the file is new
        if not file_exists:
            log_writer.writerow(['Timestamp', 'HDENS_NAME', 'agglo_Id', 'uc_km2', 'gdf2_km2', 'gdf1_gdf2_km2', 
                                 'ua_km2', 'uagreen_km2', 'uagreen_urbc_km2', 'nqgreen_m2', 'green_not_covered_by_gdf1_m2',
                                  'GQA_m2', 'GNA_m2', 'prDuration'])

        # Write the log entry
        log_writer.writerow(log_entry)

def create_log_entry(val1, val2, val3, val4, val5, val6, val7, val8, val9, val10, val11, val12, processing_time):
    timestamp = datetime.now().strftime('%Y%m%d_%Hh%Mm%Ss')
    processing_duration = str(processing_time)
    return [timestamp, val1, val2, val3, val4, val5, val6, val7, val8, val9, val10, val11, val12, processing_duration]

def parallel_overlay(gdf1, gdf2, npartitions=10, how='intersection'):
    """
    Perform spatial overlay (intersection) using GeoPandas in parallel with Dask.
    
    Parameters:
    gdf1 (GeoDataFrame): The first GeoDataFrame to overlay.
    gdf2 (GeoDataFrame): The second GeoDataFrame to overlay.
    npartitions (int): Number of partitions to use for Dask.
    how (str): The type of overlay operation. Default is 'intersection'.
    
    Returns:
    GeoDataFrame: The result of the overlay operation.
    """
    # Convert GeoDataFrames to Dask GeoDataFrames
    gdf1_dg = dg.from_geopandas(gdf1, npartitions=npartitions)
    gdf2_dg = dg.from_geopandas(gdf2, npartitions=npartitions)

    # Perform spatial overlay (intersection) using GeoPandas in parallel with Dask
    gdf1_parts = [gdf1_dg.partitions[i].compute() for i in range(gdf1_dg.npartitions)]
    gdf2_parts = [gdf2_dg.partitions[i].compute() for i in range(gdf2_dg.npartitions)]

    # Manually overlay using GeoPandas
    overlays = [gpd.overlay(gdf1_part, gdf2_part, how=how) for gdf1_part in gdf1_parts for gdf2_part in gdf2_parts]
    result = gpd.GeoDataFrame(pd.concat(overlays, ignore_index=True))
    gc.collect()
    
    return result   

In [16]:
# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
outdata_f = os.path.join(indata_f, 'OutputData', 'test0207')
if not os.path.exists(outdata_f):
    # Create the folder if it doesn't exist
    os.makedirs(outdata_f)

# 0 PREPARE A LOG FILE FOR QC
log_file = 'log_GQA_Step1.csv'
log_path = os.path.join(outdata_f, log_file)

# Initialize Dask client
client = Client()

# 1 READ URBAN CENTRES
# Read shapefile
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG.shp')
# Read the GeoPackage file
uc = gpd.read_file(uc_file_path)


# 2 READ NOISE DATA
# Load agglomerations delineations
agls_file_path = os.path.join(indata_f, 'NoiseData', 'DF1_5_Agglomerations_20240429.gpkg')

# Read the GeoPackage file
agls = gpd.read_file(agls_file_path, layer = 'dbo.DF15_AgglomerationSource_Valid_LatestDelivery')

cities_ls = ['Kaunas', 'København', 'Girona', 'Bordeaux']
# Loop through test cities
for cityLocalName in cities_ls[:1]:
    print(str(cityLocalName))
    start_time = datetime.now()
    print(str(start_time))

    uc_city = uc.query(f'HDENS_NAME == "{cityLocalName}"')
    ctry_code = uc_city.CNTR_CODE.values.astype(str)[0]
    cityLocalName_unicode = unidecode(cityLocalName)
    
    output_path = os.path.join(outdata_f, f'{ctry_code}_{cityLocalName_unicode}_GQA_centroids.shp')
    if not os.path.exists(output_path):
        print(f'loading aglomeration city {cityLocalName_unicode}')
        agl_city = agls.query(f'agglomerationName_localName == "{cityLocalName}"')
        # Keep only subset of columns
        agl_city = agl_city[['agglomerationId_identifier', 'agglomerationName_nameEng', 'geometry' ]]
        if agl_city.empty:
            agglomerationId_identifier = 'NotAvailable'
            print ("agglomerationId_identifier")
        else:
            agglomerationId_identifier = agl_city.agglomerationId_identifier.values.astype(str)[0]
            
            print ("agglomerationId_identifier")

            # Check noise contour maps GeoPackage file
            ncm_file_path = os.path.join(indata_f, 'NoiseData', f'Noise_20202025_export_{ctry_code}.gpkg')
            #layerName = f'dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_{ctry_code}'
            layerName = f'dbodf48_agg_noisecontours_roadsinagglomeration_lden_valid_latestdelivery_poly_{ctry_code}'
            ncm = gpd.read_file(ncm_file_path, layer=layerName)
            print ("ncm")

            # subset columns
            ncm_gdf = ncm[['category', 'geometry']]

            # Perform spatial overlay (intersection) using GeoPandas in parallel with Dask
            ncm_agl = parallel_overlay(ncm_gdf, agl_city, npartitions=10, how='intersection')
            print("ncm_agl")

            # Aggregate the area with lower band values (quieter bands)
            ncm_agl_city = parallel_overlay(ncm_agl, agl_city, npartitions=10, how='union')
            ncm_agl_city.category.fillna(0)

            # Select a subset of columns of interest
            ncm_dis = ncm_agl_city[['category', 'geometry']]
            
            # Define the list of noisy classes
            noisy_classes = ['Lden5559', 'Lden6064', 'Lden6569', 'Lden7074', 'LdenGreaterThan75']

            # Create a condition based on the category column
            condition = ncm_dis['category'].isin(noisy_classes)  # Replace 'category_column' with the actual column name

            # Specify the condition and create a new category column based on the condition
            ncm_dis['noisy'] = 0
            ncm_dis.loc[condition, 'noisy'] = 1
            ncm_dis = ncm_dis[['noisy', 'geometry']]
            ncm_dis_dg = dg.from_geopandas(ncm_dis, npartitions=10)
            ncm_dis = ncm_dis_dg.dissolve(by='noisy').compute().reset_index()
            print ("ncm_dis")

            # 3 READ UA DATA        
            # Load GeoPackage info
            data_f = r'A:\Copernicus\UrbanAtlas\UrbanAtlas\UA2018'
            ctry_code = uc_city.CNTR_CODE.values.astype(str)[0] 
            city_unicodeName_upper = unidecode(cityLocalName).upper()
            folder_path = glob.glob(os.path.join(data_f, f'{ctry_code}*{city_unicodeName_upper}*'))
            ua_file_path =  glob.glob(os.path.join(folder_path[0], 'Data', f'{ctry_code}*{city_unicodeName_upper}*.gpkg'))
            layers_ls = fiona.listlayers(ua_file_path[0])
            print ("layers_ls")

            # Read the GeoPackage file
            ua = gpd.read_file(ua_file_path[0], layer= layers_ls[0])
            print ("ua")

            # Select 'green' classes
            uagreen = ua.query('code_2018 == "14100" or code_2018 == "31000"')
            
            # 4 SELECT UA INTERSECTING UC
            # Perform spatial overlay (intersection)
            uagreen_urbc = parallel_overlay(uagreen, uc_city, npartitions=10, how='intersection')

            # 5 IDENTIFY GREEN AREAS EXCLUDED (NOT COVERED BY NCM)
            # Perform spatial overlay (intersection)
            nqgreen = parallel_overlay(uagreen_urbc, ncm_dis, npartitions=10, how='intersection')
            not_covered = uagreen_urbc.geometry.difference(uagreen_urbc.geometry.intersection(nqgreen.geometry.unary_union))
            # Filter out empty polygons(not empty polygons)
            green_not_covered_by_ncm = not_covered[~not_covered.is_empty]

            # save to shapefile
            file_path = os.path.join(outdata_f, f'{ctry_code}_{cityLocalName_unicode}_green_not_covered_by_ncm.shp')
            green_not_covered_by_ncm.to_file(file_path, driver='ESRI Shapefile')
            print ("green_not_covered_by_ncm")

            # 6 IDENTIFY QUIET/NOISY AREAS
            ## for statistics need to calculate area again
            # Calculate the area for each geometry and create a new column 'area'
            nqgreen['area_m2'] = nqgreen['geometry'].area
            nqgreen['area_ha'] = round(nqgreen['area_m2']* 0.0001,2)
            nqgreen['area_km2'] = round(nqgreen['area_ha']* 0.01,2)
            nqgreen_area = nqgreen.groupby(['code_2018', 'noisy'])['area_m2'].sum().reset_index()
            nqgreen_area['area_ha'] = round(nqgreen_area['area_m2']* 0.0001,2)
            nqgreen_area['area_km2'] = round(nqgreen_area['area_ha']* 0.01,2)

            # 7 EXPORT GREEN QUIET AREAS (GQA)
            nqgreen = nqgreen[['country', 'fua_name', 'fua_code', 'HDENS_2011', 'code_2018', 'class_2018', 'noisy',  'area_m2', 'area_ha', 'area_km2', 'geometry']]
            GQA = nqgreen.query('noisy == 0')
            GNA = nqgreen.query('noisy == 1')

            # Export to shapefile
            file_path = os.path.join(outdata_f, f'{ctry_code}_{cityLocalName_unicode}_GQA.shp')
            GQA.to_file(file_path, driver='ESRI Shapefile')
            print ("GQA")

            # 8 CREATE CENTROIDS FOR GQA POLYGONS
            # Create a new GeoDataFrame with centroids as points
            GQA_pts = gpd.GeoDataFrame(geometry=GQA['geometry'].centroid)
            GQA_pts['oid'] = GQA.index
            GQA_pts['fua_name'] = GQA.fua_name
            GQA_pts['fua_code'] = GQA.fua_code
            GQA_pts['HDENS_2011'] = GQA.HDENS_2011

            # Export to shapefile
            file_path = os.path.join(outdata_f, f'{ctry_code}_{cityLocalName_unicode}_GQA_centroids.shp')
            GQA_pts.to_file(file_path, driver='ESRI Shapefile')

            print ("GQA_pts")
    
            # Calculate the duration
            end_time = datetime.now()
            processing_time = end_time - start_time

            print ("str(processing_time)")
            
            ## write output values into log file
            uc_km2 = round(uc_city.area.sum()/1000000,2)
            agl_city_km2 = round(agl_city.area.sum()/1000000,2)
            ncm_agl_city_km2 = round(ncm_agl_city.area.sum()/1000000,2)
            ua_km2 = round(ua.area.sum()/1000000,2)
            uagreen_km2 = round(uagreen.area.sum()/1000000,2)
            uagreen_urbc_km2 = round(uagreen_urbc.area.sum()/1000000,2)
            nqgreen_m2 = round(nqgreen.area.sum(),2)
            green_not_covered_by_ncm_m2 = round(green_not_covered_by_ncm.area.sum(),2)
            GQA_m2 = round(GQA.area.sum(),2)
            GNA_m2 = round(GNA.area.sum(),2)
            processing_duration = str(processing_time)

            log_entry = create_log_entry(cityLocalName, agglomerationId_identifier, uc_km2, agl_city_km2, 
                                    ncm_agl_city_km2,ua_km2, uagreen_km2, uagreen_urbc_km2, nqgreen_m2, 
                                    green_not_covered_by_ncm_m2, GQA_m2, GNA_m2, processing_time)
            write_log(log_path, log_entry)
            # Clean up intermediate variables to free memory
            del agl_city, ncm, ncm_agl, ncm_agl_city, ncm_dis, ua, uagreen, uagreen_urbc, nqgreen, green_not_covered_by_ncm, GQA, GNA, GQA_pts

            # Shut down the Dask client
            client.shutdown()


            

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61281 instead
  warnings.warn(


Kaunas
2024-07-02 13:24:26.252825
loading aglomeration city Kaunas
agglomerationId_identifier
ncm
ncm_agl


Q:\UserTemp\ubach\AppData\Local\Temp\63\ipykernel_52220\1119770620.py:45: UserWarning: `keep_geom_type=True` in overlay resulted in 3346 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  overlays = [gpd.overlay(gdf1_part, gdf2_part, how=how) for gdf1_part in gdf1_parts for gdf2_part in gdf2_parts]
Q:\UserTemp\ubach\AppData\Local\Temp\63\ipykernel_52220\1119770620.py:45: UserWarning: `keep_geom_type=True` in overlay resulted in 1128 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  overlays = [gpd.overlay(gdf1_part, gdf2_part, how=how) for gdf1_part in gdf1_parts for gdf2_part in gdf2_parts]
Q:\UserTemp\ubach\AppData\Local\Temp\63\ipykernel_52220\1119770620.py:45: UserWarning: `keep_geom_type=True` in overlay resulted in 480 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  overlays = [gpd.overl

ncm_dis
layers_ls
ua
green_not_covered_by_ncm
GQA
GQA_pts
str(processing_time)


In [21]:
# save to shapefile
file_path = os.path.join(outdata_f, f'{ctry_code}_{cityLocalName_unicode}_ncm_dis.shp')
ncm_dis.to_file(file_path, driver='ESRI Shapefile')

NameError: name 'ncm_dis' is not defined

In [13]:
client.shutdown()

In [18]:
pip install pygeos

     ---------------------------------------- 1.4/1.4 MB 3.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [25]:
import geopandas as gpd
import pygeos

gpd.options.use_pygeos = False

In [26]:
cities_ls = ['Kaunas', 'København', 'Girona', 'Bordeaux']
# Loop through test cities
for cityLocalName in cities_ls[3:4]:
    print(str(cityLocalName))
    start_time = datetime.now()
    print(str(start_time))
    uc_city = uc.query(f'HDENS_NAME == "{cityLocalName}"')
    ctry_code = uc_city.CNTR_CODE.values.astype(str)[0]
    cityLocalName_unicode = unidecode(cityLocalName)
    output_path = os.path.join(outdata_f, f'{ctry_code}_{cityLocalName_unicode}_GQA_centroids.shp')
    if not os.path.exists(output_path):
        print(f'loading aglomeration city {cityLocalName_unicode}')
        agl_city = agls.query(f'agglomerationName_localName == "{cityLocalName}"')
        # Keep only subset of columns
        agl_city = agl_city[['agglomerationId_identifier', 'agglomerationName_nameEng', 'geometry' ]]
        if agl_city.empty:
            agglomerationId_identifier = 'NotAvailable'
            print ("agglomerationId_identifier")
        else:
            agglomerationId_identifier = agl_city.agglomerationId_identifier.values.astype(str)[0]
            
            print ("agglomerationId_identifier")

            # Check noise contour maps GeoPackage file
            ncm_file_path = os.path.join(indata_f, 'NoiseData', f'Noise_20202025_export.gpkg')
            #ncm_file_path = os.path.join(indata_f, 'NoiseData', f'Noise_20202025_export_{ctry_code}.gpkg')
            layerName = f'dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_{ctry_code}'
            #layerName = f'dbodf48_agg_noisecontours_roadsinagglomeration_lden_valid_latestdelivery_poly_{ctry_code}'
            ncm = gpd.read_file(ncm_file_path, layer=layerName)
            print ("ncm")

            # subset columns
            ncm_gdf = ncm[['category', 'geometry']]

            # Perform spatial overlay (intersection) using GeoPandas in parallel with Dask
            ncm_agl = parallel_overlay(ncm_gdf, agl_city, npartitions=10, how='intersection')
            print("ncm_agl")

            # Aggregate the area with lower band values (quieter bands)
            ncm_agl_city = parallel_overlay(ncm_agl, agl_city, npartitions=10, how='union')
            ncm_agl_city.category.fillna(0)

            # Select a subset of columns of interest
            ncm_dis = ncm_agl_city[['category', 'geometry']]
            
            # Define the list of noisy classes
            noisy_classes = ['Lden5559', 'Lden6064', 'Lden6569', 'Lden7074', 'LdenGreaterThan75']

            # Create a condition based on the category column
            condition = ncm_dis['category'].isin(noisy_classes)  # Replace 'category_column' with the actual column name

            # Specify the condition and create a new category column based on the condition
            ncm_dis['noisy'] = 0
            ncm_dis.loc[condition, 'noisy'] = 1
            ncm_dis = ncm_dis[['noisy', 'geometry']]
            ncm_dis_dg = dg.from_geopandas(ncm_dis, npartitions=10)
            ncm_dis = ncm_dis_dg.dissolve(by='noisy').compute().reset_index()
            print ("ncm_dis")

            # 3 READ UA DATA        
            # Load GeoPackage info
            data_f = r'A:\Copernicus\UrbanAtlas\UrbanAtlas\UA2018'
            ctry_code = uc_city.CNTR_CODE.values.astype(str)[0] 
            city_unicodeName_upper = unidecode(cityLocalName).upper()
            folder_path = glob.glob(os.path.join(data_f, f'{ctry_code}*{city_unicodeName_upper}*'))
            ua_file_path =  glob.glob(os.path.join(folder_path[0], 'Data', f'{ctry_code}*{city_unicodeName_upper}*.gpkg'))
            layers_ls = fiona.listlayers(ua_file_path[0])
            print ("layers_ls")

            # Read the GeoPackage file
            ua = gpd.read_file(ua_file_path[0], layer= layers_ls[0])
            print ("ua")

            # Select 'green' classes
            uagreen = ua.query('code_2018 == "14100" or code_2018 == "31000"')
            
            # 4 SELECT UA INTERSECTING UC
            # Perform spatial overlay (intersection)
            uagreen_urbc = parallel_overlay(uagreen, uc_city, npartitions=10, how='intersection')

            # 5 IDENTIFY GREEN AREAS EXCLUDED (NOT COVERED BY NCM)
            # Perform spatial overlay (intersection)
            nqgreen = parallel_overlay(uagreen_urbc, ncm_dis, npartitions=10, how='intersection')
            not_covered = uagreen_urbc.geometry.difference(uagreen_urbc.geometry.intersection(nqgreen.geometry.unary_union))
            # Filter out empty polygons(not empty polygons)
            green_not_covered_by_ncm = not_covered[~not_covered.is_empty]

            # save to shapefile
            file_path = os.path.join(outdata_f, f'{ctry_code}_{cityLocalName_unicode}_green_not_covered_by_ncm.shp')
            green_not_covered_by_ncm.to_file(file_path, driver='ESRI Shapefile')
            print ("green_not_covered_by_ncm")

            # 6 IDENTIFY QUIET/NOISY AREAS
            ## for statistics need to calculate area again
            # Calculate the area for each geometry and create a new column 'area'
            nqgreen['area_m2'] = nqgreen['geometry'].area
            nqgreen['area_ha'] = round(nqgreen['area_m2']* 0.0001,2)
            nqgreen['area_km2'] = round(nqgreen['area_ha']* 0.01,2)
            nqgreen_area = nqgreen.groupby(['code_2018', 'noisy'])['area_m2'].sum().reset_index()
            nqgreen_area['area_ha'] = round(nqgreen_area['area_m2']* 0.0001,2)
            nqgreen_area['area_km2'] = round(nqgreen_area['area_ha']* 0.01,2)

            # 7 EXPORT GREEN QUIET AREAS (GQA)
            nqgreen = nqgreen[['country', 'fua_name', 'fua_code', 'HDENS_2011', 'code_2018', 'class_2018', 'noisy',  'area_m2', 'area_ha', 'area_km2', 'geometry']]
            GQA = nqgreen.query('noisy == 0')
            GNA = nqgreen.query('noisy == 1')

            # Export to shapefile
            file_path = os.path.join(outdata_f, f'{ctry_code}_{cityLocalName_unicode}_GQA.shp')
            GQA.to_file(file_path, driver='ESRI Shapefile')
            print ("GQA")

            # 8 CREATE CENTROIDS FOR GQA POLYGONS
            # Create a new GeoDataFrame with centroids as points
            GQA_pts = gpd.GeoDataFrame(geometry=GQA['geometry'].centroid)
            GQA_pts['oid'] = GQA.index
            GQA_pts['fua_name'] = GQA.fua_name
            GQA_pts['fua_code'] = GQA.fua_code
            GQA_pts['HDENS_2011'] = GQA.HDENS_2011

            # Export to shapefile
            file_path = os.path.join(outdata_f, f'{ctry_code}_{cityLocalName_unicode}_GQA_centroids.shp')
            GQA_pts.to_file(file_path, driver='ESRI Shapefile')

            print ("GQA_pts")
    
            # Calculate the duration
            end_time = datetime.now()
            processing_time = end_time - start_time

            print ("str(processing_time)")
            
            ## write output values into log file
            uc_km2 = round(uc_city.area.sum()/1000000,2)
            agl_city_km2 = round(agl_city.area.sum()/1000000,2)
            ncm_agl_city_km2 = round(ncm_agl_city.area.sum()/1000000,2)
            ua_km2 = round(ua.area.sum()/1000000,2)
            uagreen_km2 = round(uagreen.area.sum()/1000000,2)
            uagreen_urbc_km2 = round(uagreen_urbc.area.sum()/1000000,2)
            nqgreen_m2 = round(nqgreen.area.sum(),2)
            green_not_covered_by_ncm_m2 = round(green_not_covered_by_ncm.area.sum(),2)
            GQA_m2 = round(GQA.area.sum(),2)
            GNA_m2 = round(GNA.area.sum(),2)
            processing_duration = str(processing_time)

            log_entry = create_log_entry(cityLocalName, agglomerationId_identifier, uc_km2, agl_city_km2, 
                                    ncm_agl_city_km2,ua_km2, uagreen_km2, uagreen_urbc_km2, nqgreen_m2, 
                                    green_not_covered_by_ncm_m2, GQA_m2, GNA_m2, processing_time)
            write_log(log_path, log_entry)
            # Clean up intermediate variables to free memory
            del agl_city, ncm, ncm_agl, ncm_agl_city, ncm_dis, ua, uagreen, uagreen_urbc, nqgreen, green_not_covered_by_ncm, GQA, GNA, GQA_pts

            # Shut down the Dask client
            client.shutdown()



Bordeaux
2024-07-02 16:51:46.360926
loading aglomeration city Bordeaux
agglomerationId_identifier


2024-07-02 16:51:53,605 - distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)
2024-07-02 16:52:17,175 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-07-02 16:52:18,538 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-07-02 16:52:22,340 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-07-02 16:52:24,539 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
2024-07-02 16:52:26,542 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
2024-07-02 16:52:28,915 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
2024-07-02 16:52:31,340 - distributed.utils_perf - WARNING - full garbage collections took

ncm


2024-07-02 16:55:29,065 - distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)


ncm_agl


Q:\UserTemp\ubach\AppData\Local\Temp\63\ipykernel_52220\1119770620.py:45: UserWarning: `keep_geom_type=True` in overlay resulted in 9923 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  overlays = [gpd.overlay(gdf1_part, gdf2_part, how=how) for gdf1_part in gdf1_parts for gdf2_part in gdf2_parts]
Q:\UserTemp\ubach\AppData\Local\Temp\63\ipykernel_52220\1119770620.py:45: UserWarning: `keep_geom_type=True` in overlay resulted in 3905 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  overlays = [gpd.overlay(gdf1_part, gdf2_part, how=how) for gdf1_part in gdf1_parts for gdf2_part in gdf2_parts]
Q:\UserTemp\ubach\AppData\Local\Temp\63\ipykernel_52220\1119770620.py:45: UserWarning: `keep_geom_type=True` in overlay resulted in 2721 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  overlays = [gpd.over

ncm_dis
layers_ls
ua
green_not_covered_by_ncm
GQA
GQA_pts
str(processing_time)


In [23]:
import fiona

gpkg_path =  os.path.join(indata_f, 'NoiseData', f'Noise_20202025_export.gpkg')
layers = fiona.listlayers(gpkg_path)

for layer_name in layers:
    print(layer_name)



dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_ES
dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_FR
dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_CY
dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_BE
dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_CZ
dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_DK
dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_EE
dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_FI
dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_IE
dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_LT
dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_LV
dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_SE
dbo.DF15_Agglome

In [22]:
# Check noise contour maps GeoPackage file
#ncm_file_path = os.path.join(indata_f, 'NoiseData', f'Noise_20202025_export_{ctry_code}.gpkg')
ncm_file_path = os.path.join(indata_f, 'NoiseData', f'Noise_20202025_export.gpkg')
#layerName = f'dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_{ctry_code}'
layerName = f'dbodf48_agg_noisecontours_roadsinagglomeration_lden_valid_latestdelivery_poly_{ctry_code}'
ncm = gpd.read_file(ncm_file_path, layer=layerName)
print ("ncm")

ValueError: Null layer: 'dbodf48_agg_noisecontours_roadsinagglomeration_lden_valid_latestdelivery_poly_FR'